In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from tqdm import trange
import re
from lmfit.models import GaussianModel, StepModel, Model
from cn_tools.cns_fit import cns_pc_from_file
from cn_tools.cns_fit import mu_from_gauss
from cn_tools.cns_fit import sigma_from_cns
from cn_tools.cns_fit import Z_from_cns
from cn_tools.cns_fit import compute_local_contacts
from cn_tools.cns_fit import compute_nearest_neighbours
from cn_tools.cns_fit import compute_mean_sigma
from cn_tools.cns_fit import approximate_diameter
from cn_tools.data_processing import cut_df

In [2]:
directories = ['../Data/preprocessed/VF_005_analysis/', '../Data/preprocessed/VF_006_analysis/',
               '../Data/preprocessed/VF_007_analysis/', '../Data/preprocessed/VF_008_analysis/']

In [3]:
def compute_sigmas(directory):
    '''Iterates through a raw directory and saves all processed csv files in another directory called preprocessed'''
    
    #define save directory
    save_file = '../Data/preprocessed/sigma.csv'
    df_sigma = pd.read_csv(save_file, index_col=0)
        
    #create list of all minkowski file folders to evaluate
    files = [directory + file for file in os.listdir(directory) if 'tomo' in file]
    
    #for folder in folders:
    for i in trange(len(files)):
        
        file_name = files[i]
        #index has the form 'XXX_YYY'
        index = file_name[-11:-4]
        interval, cns, pc = cns_pc_from_file(file_name)
        mu = mu_from_gauss(interval, pc)
        sigma = sigma_from_cns(interval, cns, mu)
        df_sigma.loc[index, 'sigma'] = sigma
        
    df_sigma.to_csv(save_file)

In [8]:
def contacts_and_neighbours(directory):
    #final directory
    final_dir = re.sub('preprocessed', 'final', directory)
    if not os.path.isdir(final_dir):
        os.mkdir(final_dir)
    #compute mean sigmas
    mean_sigmas = {res:compute_mean_sigma(resolution=res) for res in [20,30]}
    #pick relevant sigma according to directory name
    sigma = mean_sigmas[approximate_diameter(directory)]
    
    files = [directory + file for file in os.listdir(directory) if 'tomo' in file]
    
    for i in trange(len(files)):
        file_name = files[i]
        poly_file = re.sub('preprocessed', 'raw', file_name)
        poly_file = re.sub('.csv', '.poly', poly_file)
        final_file = re.sub('preprocessed', 'final', file_name)
        interval, cns, pc = cns_pc_from_file(file_name)
        mu = mu_from_gauss(interval, pc)
        Z = Z_from_cns(interval, cns, mu, sigma)
        compute_local_contacts(file_name, Z)
        compute_nearest_neighbours(file_name, poly_file)
        
        df = pd.read_csv(file_name, index_col = 0)
        df = cut_df(df)
        df.to_csv(final_file)

In [4]:
#directories  = ['../Data/preprocessed/VF_008_analysis_ref_centroid/']
for directory in directories:
    compute_sigmas(directory)

100%|██████████| 308/308 [02:39<00:00,  1.93it/s]


In [9]:
for directory in directories:
    contacts_and_neighbours(directory)

100%|██████████| 308/308 [14:32<00:00,  2.83s/it]
